In [81]:
# merge_all.csv 파일을 사용하여 "가격 (원)" 컬럼의 값을 예측하는 모델을 만든다

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import keras
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.layers import PReLU
from keras.layers import ELU
from keras.layers import ThresholdedReLU
from keras.layers import Softmax
from keras.layers import ReLU
from keras.layers import GaussianNoise
from keras.layers import GaussianDropout
from keras.layers import AlphaDropout
from keras.layers import ActivityRegularization
from keras.layers import Masking
from keras.layers import SpatialDropout1D

# r2_score import
from sklearn.metrics import r2_score





In [36]:
# 데이터 불러오기
df = pd.read_csv('data/merge_all/merge_all.csv')

# 데이터 전처리
# "거래년월일" 컬럼을 인덱스로 설정
df = df.set_index('거래년월일')
# "평균" 컬럼을 배추가격으로 설정
df = df.rename(columns={'평균':'배추가격'})
df.head()

,자동차용 경유 가격 (원),전월비(%),배추가격,거래량
거래년월일,,,,
2000-01-01,583.35,4.0,5600.0,102285.0
2000-01-02,583.35,4.0,5600.0,102285.0
2000-01-03,583.35,4.0,5600.0,102285.0
2000-01-04,583.35,4.0,5600.0,715338.0
2000-01-05,583.35,4.0,5620.0,340568.0


In [42]:
# df 를 시퀀스 데이터로 변환
# 1. 데이터를 7일 단위로 묶는다

# 7일 단위로 묶는 함수
def make_sequence_data(data, window_size):
    x, y = [], []
    for i in range(len(data) - window_size):
        group = data.iloc[i:(i+window_size)]
        x.append(group.values)
        y.append(data.iloc[i+window_size][0])
    return np.array(x), np.array(y)

# 7일 단위로 묶는다
window_size = 7
x, y = make_sequence_data(df, window_size)
print(x.shape, y.shape)

# 2. 데이터를 학습용 데이터와 테스트용 데이터로 나눈다
# 학습용 데이터와 테스트용 데이터를 7:3으로 나눈다
train_size = int(len(y) * 0.7)
test_size = len(y) - train_size
train_x, test_x = np.array(x[0:train_size]), np.array(x[train_size:len(x)])
train_y, test_y = np.array(y[0:train_size]), np.array(y[train_size:len(y)])
print(train_x.shape, test_x.shape)

# 3. 데이터를 정규화한다
# 정규화 함수
def minmax_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

# 정규화
train_x = minmax_scaler(train_x)
train_y = minmax_scaler(train_y)
test_x = minmax_scaler(test_x)
test_y = minmax_scaler(test_y)

print(train_x.shape)
print(test_x.shape)

# 4. 데이터를 LSTM 모델에 입력할 수 있는 형태로 변환한다
# LSTM 모델에 입력할 수 있는 형태로 변환
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1], 4)
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1], 4)
print(train_x.shape)
print(test_x.shape)


(8363, 7, 4) (8363,)
(5854, 7, 4) (2509, 7, 4)
(5854, 7, 4)
(2509, 7, 4)
(5854, 7, 4)
(2509, 7, 4)


In [44]:
model = Sequential()
# LSTM 모델을 만든다
model.add(LSTM(32, input_shape=(window_size, 4), return_sequences=True))
# maxpooling
model.add(MaxPooling1D(pool_size=2))
# flatten
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()





Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 7, 32)             4736      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 3, 32)            0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 96)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 97        
                                                                 
Total params: 4,833
Trainable params: 4,833
Non-trainable params: 0
_________________________________________________________________


In [65]:
#  학습을 진행한다
# 학습을 진행한다
early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)
model.fit(train_x, train_y, epochs=500, batch_size=32, verbose=1, callbacks=[early_stop])


Epoch 1/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.2624e-05
Epoch 2/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.4140e-05
Epoch 3/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.2807e-05
Epoch 4/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.4078e-05
Epoch 5/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.1804e-05
Epoch 6/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.3781e-05
Epoch 7/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.1900e-05
Epoch 8/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.0383e-05
Epoch 9/1000
183/183 [==============================] - 2s 13ms/step - loss: 6.3917e-05
Epoch 10/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.3109e-05
Epoch 11/1000
183/183 [==============================] - 2s 12ms/step - loss: 6.4423e-05
Epoch 12/1000
183/183 [=======

In [66]:
# 모델저장
model.save('model/bestmodel.h5')

In [78]:
# 모델 성능을 평가
# 학습용 데이터로 모델을 평가
train_predict = model.predict(train_x)
test_predict = model.predict(test_x)

# 정규화된 값을 원래 값으로 되돌린다
train_predict = train_predict * (np.max(df['배추가격']) - np.min(df['배추가격'])) + np.min(df['배추가격'])
train_y = train_y * (np.max(df['배추가격']) - np.min(df['배추가격'])) + np.min(df['배추가격'])
test_predict = test_predict * (np.max(df['배추가격']) - np.min(df['배추가격'])) + np.min(df['배추가격'])

test_y = test_y * (np.max(df['배추가격']) - np.min(df['배추가격'])) + np.min(df['배추가격'])

# 학습용 데이터로 모델을 평가
trainScore = math.sqrt(mean_squared_error(train_y, train_predict))
print('Train Score: %.2f RMSE' % (trainScore))
# 테스트용 데이터로 모델을 평가


79/79 [==============================] - 0s 4ms/step
Train Score: 1066797094556173824.00 RMSE


In [84]:
test_predict

array([[0.09139802],
       [0.09122159],
       [0.0911039 ],
       ...,
       [0.7979276 ],
       [0.79751027],
       [0.7987635 ]], dtype=float32)